In [8]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [12]:
stops = gpd.read_file('zip://../../rawdata/bus/bus_stops_nyc_nov2018.zip')
routes = gpd.read_file('zip://../../rawdata/bus/bus_routes_nyc_nov2018.zip')

In [14]:
stops

,stop_id,stop_name,stop_lat,stop_lon,GEOID,NAMELSAD,geometry
0,100294,BRUCKNER BL/TREMONT AV,40.833150,-73.827630,36005,Bronx County,POINT (1031949.407765885 242863.2320205418)
1,100296,BRUCKNER BL/KEARNEY AV,40.839142,-73.824818,36005,Bronx County,POINT (1032723.196746612 245047.8794435479)
2,100298,MACDONOUGH PL/COUNTRY CLUB,40.842607,-73.825077,36005,Bronx County,POINT (1032649.010098742 246310.1628801106)
3,101716,BAYCHESTER AV/ALDRICH ST,40.870415,-73.832488,36005,Bronx County,POINT (1030579.107005887 256437.6590170595)
4,101717,BAYCHESTER AV/DONIZETTI PL,40.876173,-73.833697,36005,Bronx County,POINT (1030240.749229186 258534.8849715356)
5,101718,BAYCHESTER AV/DARROW PL,40.878532,-73.832806,36005,Bronx County,POINT (1030485.512674163 259394.8299751458)
6,101723,CO-OP CITY BL/CARVER LOOP #2,40.879393,-73.825075,36005,Bronx County,POINT (1032622.799591572 259712.7008976294)
7,101724,CO-OP CITY BL/BELLAMY LOOP N,40.873841,-73.824577,36005,Bronx County,POINT (1032764.564497987 257690.166214562)
8,101725,CO-OP CITY BL/BELLAMY LOOP S,40.872141,-73.824749,36005,Bronx County,POINT (1032718.235806853 257070.6951215542)
9,101729,HUTCHINSON RVR PKY/EINSTEIN LOOP N,40.865644,-73.823627,36005,Bronx County,POINT (1033033.30702169 254704.2108517577)
